In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Input, Add, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
## Uncomment to convert images to ELA

# from PIL import Image
# import os
# from pylab import *
# import re
# from PIL import Image, ImageChops, ImageEnhance

# def convert_to_ela_image(path, quality):
#     filename = path
#     resaved_filename = filename.split('.')[0] + '.resaved.jpg'
#     ELA_filename = filename.split('.')[0] + '.ela.png'
    
#     im = Image.open(filename).convert('RGB')
#     im.save(resaved_filename, 'JPEG', quality=quality)
#     resaved_im = Image.open(resaved_filename)
    
#     ela_im = ImageChops.difference(im, resaved_im)
    
#     extrema = ela_im.getextrema()
#     max_diff = max([ex[1] for ex in extrema])
#     if max_diff == 0:
#         max_diff = 1
#     scale = 255.0 / max_diff
    
#     ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    
#     os.remove(resaved_filename)
    
#     return ela_im

# for directory in (r"data\CASIA\CASIA_rgb\Au", r"data\CASIA\CASIA_rgb\Tp"):
#     for filename in os.listdir(directory):
#         if filename.endswith(".jpg"):
#             file_path = os.path.join(directory, filename)
#             pil_image = convert_to_ela_image(file_path, 90)
#             pil_image.save("data\CASIA\CASIA_ela\" + directory.split("\\")[-1] + "\\" + filename.split(".")[0] + ".png", format="PNG")

In [3]:
batch_size = 32
generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, rescale = 1./255, horizontal_flip = True)


train_generator = generator.flow_from_directory(r"data\CASIA\CASIA_ela",
                                          target_size = (128,128),
                                          class_mode = "categorical", 
                                          interpolation = "nearest",
                                          batch_size = batch_size,
                                          subset = "training")

val_generator = generator.flow_from_directory(r"data\CASIA\CASIA_ela",
                                          target_size = (128,128),
                                          class_mode = "categorical",
                                          interpolation = "nearest",
                                          batch_size = batch_size,
                                          subset = "validation")

Found 10049 images belonging to 2 classes.
Found 2511 images belonging to 2 classes.


In [4]:
def ELA_CNN(input_shape=(128,128,3)):
    X_input = Input(input_shape, name="X")
    ###################################################
    X = Conv2D(32, (5,5), padding="valid", activation=tf.nn.leaky_relu, name="conv1_1")(X_input)
    X = MaxPooling2D(pool_size=(2,2), strides=(2,2), name="max_pool1_1")(X)
    
    X = Conv2D(32, (5,5), padding="valid", activation=tf.nn.leaky_relu, name="conv1_2")(X)
    X = MaxPooling2D(pool_size=(2,2), strides=(2,2), name="max_pool1_2")(X)
    X = Dropout(0.25, name="dropout1_1")(X)
    ##########################################################
    X = Flatten()(X)
    X = Dense(256, activation=tf.nn.leaky_relu, name="fc1", kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005))(X)
    
    X = Dropout(0.5, name="dropout2")(X)
    X = Dense(2, activation="softmax", name="fc2")(X)
    
    model = tf.keras.Model(inputs=X_input, outputs=X)
    
    return model

In [5]:
model = ELA_CNN()

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
X (InputLayer)               [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 124, 124, 32)      2432      
_________________________________________________________________
max_pool1_1 (MaxPooling2D)   (None, 62, 62, 32)        0         
_________________________________________________________________
conv1_2 (Conv2D)             (None, 58, 58, 32)        25632     
_________________________________________________________________
max_pool1_2 (MaxPooling2D)   (None, 29, 29, 32)        0         
_________________________________________________________________
dropout1_1 (Dropout)         (None, 29, 29, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 26912)             0     

In [7]:
# model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9, decay=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath=r"data\checkpoints\tampering_detection\\",
                                                               save_weights_only=True)

model.fit(train_generator,
          batch_size=batch_size,
          epochs=30,
          validation_data=val_generator,
          steps_per_epoch=10049//batch_size,
          validation_steps=2511//batch_size
         )

Epoch 1/30
314/314 [==============================] - 50s 158ms/step - loss: 0.6688 - accuracy: 0.6722 - val_loss: 0.6378 - val_accuracy: 0.6815
Epoch 2/30
314/314 [==============================] - 51s 162ms/step - loss: 0.5899 - accuracy: 0.7157 - val_loss: 0.6537 - val_accuracy: 0.6803
Epoch 3/30
314/314 [==============================] - 48s 152ms/step - loss: 0.5903 - accuracy: 0.7206 - val_loss: 0.6944 - val_accuracy: 0.6342
Epoch 4/30
314/314 [==============================] - 44s 140ms/step - loss: 0.5929 - accuracy: 0.7358 - val_loss: 0.6689 - val_accuracy: 0.6803
Epoch 5/30
314/314 [==============================] - 43s 136ms/step - loss: 0.6120 - accuracy: 0.7503 - val_loss: 0.7130 - val_accuracy: 0.6671
Epoch 6/30
314/314 [==============================] - 43s 137ms/step - loss: 0.6262 - accuracy: 0.7609 - val_loss: 0.7002 - val_accuracy: 0.7079
Epoch 7/30
314/314 [==============================] - 45s 143ms/step - loss: 0.6215 - accuracy: 0.7797 - val_loss: 0.6878 - val_ac

In [11]:
# model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9, decay=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath=r"data\checkpoints\tampering_detection\\",
                                                               save_weights_only=True)

model.fit(train_generator,
          batch_size=batch_size,
          epochs=10,
          validation_data=val_generator,
          steps_per_epoch=10049//batch_size,
          validation_steps=2511//batch_size,
          callbacks = [mc]
         )

Epoch 1/10
314/314 [==============================] - 49s 157ms/step - loss: 0.4193 - accuracy: 0.9577 - val_loss: 0.7904 - val_accuracy: 0.8269
Epoch 2/10
314/314 [==============================] - 49s 157ms/step - loss: 0.3840 - accuracy: 0.9630 - val_loss: 0.8000 - val_accuracy: 0.8225
Epoch 3/10
314/314 [==============================] - 47s 150ms/step - loss: 0.3579 - accuracy: 0.9681 - val_loss: 0.7778 - val_accuracy: 0.8237
Epoch 4/10
314/314 [==============================] - 46s 148ms/step - loss: 0.3374 - accuracy: 0.9729 - val_loss: 0.7381 - val_accuracy: 0.8365
Epoch 5/10
314/314 [==============================] - 43s 136ms/step - loss: 0.3116 - accuracy: 0.9781 - val_loss: 0.7398 - val_accuracy: 0.8361
Epoch 6/10
314/314 [==============================] - 43s 136ms/step - loss: 0.2931 - accuracy: 0.9796 - val_loss: 0.7584 - val_accuracy: 0.8301
Epoch 7/10
314/314 [==============================] - 43s 136ms/step - loss: 0.2822 - accuracy: 0.9782 - val_loss: 0.7191 - val_ac

In [20]:
model.save("checkpoints\tampering_detection\final_tampering_model.h5")